# Restartable Jobs

## Overview

The ideal restartable job is a single job script that can be resubmitted over and over again to the job queue system. Each time the job starts, it picks up where it left off the last time and continues running until it is done. You can put all the logic necessary to do this in the hoomd python script itself, keeping the submission script simple:
```
#job.sh
python run.py
```
With a properly configured python script, qsub job.sh is all that is necessary to submit the first run, continue a previous job that exited cleanly, and continue one that was prematurely killed.

## Elements of a restartable script
A restartable needs to:
- Choose between an initial condition and the restart file when initializing.
- Write a restart file periodically.
- Set a phase for all analysis, dump, and update commands.
- Use hoomd.run_upto() to skip over time steps that were run in previous job submissions.
- Use only commands that are restart capable.
- Optionally ensure that jobs cleanly exit before the job walltime limit.

### Choose the appropriate initialization file
Let’s assume that the initial condition for the simulation is in ```init.gsd```, and ```restart.gsd``` is saved periodically as the job runs. A single ```hoomd.init.read_gsd()``` command will load the restart file if it exists, otherwise it will load the initial file. It is easiest to think about dump files, temperature ramps, etc... if ```init.gsd``` is at time step 0:
```
init.read_gsd(filename='init.gsd', restart='restart.gsd')
```




- if restart.gsd exists, load it
- else create initial state
- 